### Ingestión del archivo "movie_cast.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvio JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movies_casts_schema = StructType(fields= [
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genreId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)
])                       

In [0]:
movies_casts_df = spark.read \
        .schema(movies_casts_schema) \
        .option("multiLine", True) \
        .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json") 

#### Pase 2 - Renombrar las columnas y añadir nuevas columnas
1. "movieId" renombrar a "movie_id"
2. "personId" renombrar a "person_id"
3. "characterName" renombrar a "character_id"
4. Agregar la columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_casts_with_columns_df = add_ingestion_date(movies_casts_df) \
    .withColumnsRenamed({"movieId": "movie_id",
                         "personId": "person_id",
                         "characterName": "character_name"}) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))  


#### Paso 3 - Eliminar columnas no "requeridas"

In [0]:
from pyspark.sql.functions import col

In [0]:
movies_casts_final_df = movies_casts_with_columns_df.drop(col("genreId"), col("castOrder"))

In [0]:
display(movies_casts_with_columns_df)

#### Paso 4 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(movies_casts_final_df, "movie_silver","movies_casts", "file_date")

In [0]:
merge_condition = "tgt.movie_id = src.movie_id AND tgt.person_id = src.person_id AND tgt.file_date = src.file_date"
merge_delta_lake(movies_casts_final_df, "movie_silver","movies_casts", silver_folder_path, merge_condition, "file_date")
#movies_casts_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_casts")
#movies_casts_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_casts")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_casts
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.movies_casts;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_casts"))

In [0]:
dbutils.notebook.exit("Success")